# Mongodb en Python

In [51]:
from pymongo import MongoClient

In [15]:
# Conexión a la base de datos
mongodb_uri = "mongodb://localhost:27017"
cliente = MongoClient(mongodb_uri)

db = cliente.companies

In [16]:
# Listado de Collections
db.list_collection_names()

['companies', 'first_office']

In [17]:
# Una búsqueda devuelve un objeto Cursor
data = db.companies.find()
data

In [30]:
# Para trabajar con los datos podemos transformarlo en una lista de diccionarios
data = list(db.companies.find())
len(data)

18801

In [45]:
# Las cadenas de búsqueda son diccionarios. Ojo porque son case-sensitive
query = db.companies.find({"name":"Zoho"})
data = list(query)
len(data)

1

In [46]:
# Podemos seleccionar solo los datos que queremos recibir usando Proyección
projection = {"_id":0, "name": 1,
              "offices.latitude": 1,
              "offices.longitude": 1,
              "founded_year": 1,
              "category_code": 1,
              "number_of_employees": 1,
              "total_money_raised":1}

query = db.companies.find({"name":"Zoho"}, projection)
data=list(query)
data

[{'name': 'Zoho',
  'category_code': 'software',
  'number_of_employees': 1600,
  'founded_year': 2005,
  'total_money_raised': '$0',
  'offices': [{'latitude': 37.692934, 'longitude': -121.904945}]}]

In [76]:
# Se pueden hacer consultas mucho más complejas:
query = db.companies.find({
    "$and":[
        {"$or":[{"category_code":"games_video"},
              {"category_code":"software"},
              {"category_code":"web"},
              {"category_code":"mobile"},
              {"category_code":"photo_video"},
              {"category_code":"ecommerce"},
              {"category_code":"search"},
              {"category_code":"network_hosting"}]},
        {"founded_year":{"$gte":1999}},
        {"offices":{"$ne":[]}},
        {"offices.latitude":{"$ne":[]}},
        {"offices.latitude":{"$ne":None}},
        {"offices.longitude":{"$ne":[]}},
        {"offices.longitude":{"$ne":None}},
        {"deadpooled_year":None},
        {"number_of_employees":{"$ne":None}},
        {"total_money_raised":{"$ne":None}}
    ]}, projection)

#data=list(query)
#len(data)

# Geoqueries
La idea es crear un punto en cada oficina que represente su longitud y latitud. Luego crear un índice de Geoloc por esos puntos.

In [50]:
import pandas as pd

In [77]:
# Como el campo "offices" es una lista de diccionarios 
#   (una compañía puede tener varias oficinas con su longitud y latitud respectivas)
# es necesario hacer flatten del campo offices
df = pd.json_normalize(data = query,
                       record_path = "offices",
                       meta = ["name", "category_code", "number_of_employees", "founded_year", "total_money_raised"])
df.head()

,latitude,longitude,name,category_code,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,Geni,web,18,2006,$16.5M
1,40.746497,-74.009447,Joost,games_video,0,2006,$45M
2,37.387845,-122.055197,Plaxo,web,50,2002,$28.3M
3,37.778613,-122.395289,Powerset,search,60,2006,$22.5M
4,34.017606,-118.487267,Mahalo,web,40,2007,$21M


In [78]:
# Ahora hay que crear un solo campo con longitud y latitud
df.insert(2, "geo_loc", "n/a")

In [80]:
# Ojo, que este campo tiene que tener una Tupla con Longitud y Latitud en este orden
df["geo_loc"] = df.apply(lambda x: (x["longitude"], x["latitude"]), axis = 1)
df.head()

,latitude,longitude,geo_loc,name,category_code,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,"(-118.393064, 34.090368)",Geni,web,18,2006,$16.5M
1,40.746497,-74.009447,"(-74.0094471, 40.7464969)",Joost,games_video,0,2006,$45M
2,37.387845,-122.055197,"(-122.055197, 37.387845)",Plaxo,web,50,2002,$28.3M
3,37.778613,-122.395289,"(-122.395289, 37.778613)",Powerset,search,60,2006,$22.5M
4,34.017606,-118.487267,"(-118.487267, 34.017606)",Mahalo,web,40,2007,$21M


In [81]:
# Vamos a guardar estos datos en una colección diferente llamada "geoloc"
db.geoloc.insert_many(df.to_dict('records'))

In [82]:
# Crear el índice Geoloc
db.geoloc.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [103]:
# Mostrar en un mapa con Folium
import folium

# Centro el mapa
madrid = {"latitude":40.41682314069984, "longitude":-3.7037866499710037}
mapa=folium.Map(location=[float(madrid["latitude"]), float(madrid["longitude"])], tiles='openstreetmap', zoom_start=10)

# Pinto todas las oficinas
for i in range(len(df)):
    folium.CircleMarker([float(df.latitude[i]), float(df.longitude[i])],
                        popup=str(df.name[i]),
                        radius=1,
                        icon=folium.Icon()).add_to(mapa)

# Muestro el mapa
mapa


In [129]:
# Geoquery
geo_query = db.geoloc.find({"geo_loc":{
    "$near":{
        "$geometry":{"type":"Point", "coordinates":[madrid["longitude"], madrid["latitude"]]},
        "$maxDistance":10000
    }
} 
})

df_geo_query = pd.DataFrame(geo_query)
df_geo_query.head()

,_id,latitude,longitude,geo_loc,name,category_code,number_of_employees,founded_year,total_money_raised
0,5fbe43454568eed13357538a,40.416741,-3.703250,"[-3.70325, 40.416741]",AtelMedia,software,13,2006,$0
1,5fbe43454568eed13357599c,40.416597,-3.699267,"[-3.6992667, 40.4165967]",ASPgems,software,26,2006,$0
2,5fbe43454568eed133575517,40.412323,-3.703248,"[-3.703248, 40.412323]",Daily Flat Rental,ecommerce,2,2008,$0
3,5fbe43454568eed133575475,40.412323,-3.703248,"[-3.703248, 40.412323]",Daily Flat Rental,ecommerce,2,2008,$0
4,5fbe43454568eed133575a53,40.424459,-3.699095,"[-3.6990948, 40.4244585]",Taxi Fares app,software,5,2009,$0


In [130]:
# Pinto el mapa con el resultado

# Centro el mapa
mapa=folium.Map(location=[float(madrid["latitude"]), float(madrid["longitude"])], tiles='openstreetmap', zoom_start=10)

# Pinto todas las oficinas
for i in range(len(df_geo_query)):
    folium.CircleMarker([float(df_geo_query.latitude[i]), float(df_geo_query.longitude[i])],
                        popup=str(df_geo_query.name[i]),
                        radius=1,
                        icon=folium.Icon()).add_to(mapa)

# Muestro el mapa
mapa
